In [1]:
library(tidyverse)
library(ggpubr)
library(GGally)
library(yaml)
library(patchwork)
#library(Seurat)
#library(ggvenn)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Registered S3 method overwritten by 'GGally':
  method from   
  +.gg   ggplot2



In [2]:
#read config
config <- read_yaml(file = "../CHARM/config.yaml")

In [6]:
#FIXME:这几个数值要不要改,原来是/4*300/1G

Raw <- read_table("../stat/raw.fq.stat",col_names=F) %>% arrange(X1) %>% rowwise() %>% mutate(X2 = X2,X1 = strsplit(X1,split = "/")[[1]][3],X1 = strsplit(X1,split = "_")[[1]][1])
DNA <- read_table("../stat/dna.fq.stat",col_names=F) %>% arrange(X1) %>% rowwise()%>% mutate(X2 = X2,X1 = strsplit(X1,split = "/")[[1]][3])
RNA <- read_table("../stat/rna.fq.stat",col_names=F) %>% arrange(X1) %>% rowwise() %>% mutate(X2 = X2,X1 = strsplit(X1,split = "/")[[1]][3])



── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_character(),
  X2 = col_double()
)


── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_character(),
  X2 = col_double()
)


── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_character(),
  X2 = col_double()
)



In [7]:
raw_pairs <- read_table("../stat/raw.pairs.stat",col_names=F) %>% arrange(X1) %>% rowwise() %>% mutate(X1 = strsplit(X1,split = "/")[[1]][3])
pairs_dedup <- read_table("../stat/pairs.dedup.stat",col_names=F) %>% arrange(X1) %>% rowwise() %>% mutate(X1 = strsplit(X1,split = "/")[[1]][3])
pairs_c1 <- read_table2("../stat/pairs.c1.stat",col_names=F)%>% arrange(X1) %>%rowwise() %>% mutate(X1 = strsplit(X1,split = "/")[[1]][5]) %>% mutate(X1 = str_replace(X1,".pairs.gz",""))
pairs_c12 <- read_table2("../stat/pairs.c12.stat",col_names=F)%>% arrange(X1) %>%rowwise() %>% mutate(X1 = strsplit(X1,split = "/")[[1]][5]) %>% mutate(X1 = str_replace(X1,".pairs.gz",""))
pairs_c123 <- read_table2("../stat/pairs.c123.stat",col_names=F)%>% arrange(X1) %>%rowwise() %>% mutate(X1 = strsplit(X1,split = "/")[[1]][5]) %>% mutate(X1 = str_replace(X1,".pairs.gz",""))
inter_pairs_c123 <- read_table2(file = "../stat/inter.pairs.c123.stat",col_names = F) %>% arrange(X1) %>%rowwise() %>% mutate(X1 = strsplit(X1,split = "/")[[1]][5]) %>% mutate(X1 = str_replace(X1,".pairs.gz",""))


── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_character(),
  X2 = col_double()
)


── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_character(),
  X2 = col_double()
)

Warning message:
“`read_table2()` was deprecated in readr 2.0.0.
Please use `read_table()` instead.”

── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_character(),
  X2 = col_double()
)

Warning message:
“`read_table2()` was deprecated in readr 2.0.0.
Please use `read_table()` instead.”

── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_character(),
  X2 = col_double()
)

Warning message:
“`read_table2()` was deprecated in readr 2.0.0.
Please use `read_table()` instead.”

── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_character(),
  X2 = col_double()
)

Warning message:
“`rea

In [8]:
yperx <- read_table("../stat/yperx.stat",col_names=F) %>% arrange(X1) %>% rowwise() %>% mutate(X1 = strsplit(X1,split = "/")[[1]][2])


── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_character(),
  X2 = col_double()
)



In [9]:
#merge
stat <- Raw %>% left_join(DNA,by="X1") %>% left_join(RNA,by="X1") %>% left_join(yperx,by="X1") %>% left_join(raw_pairs,by="X1") %>% left_join(pairs_dedup,by="X1") %>% 
    left_join(pairs_c1,by="X1") %>% left_join(pairs_c12,by="X1")%>% left_join(pairs_c123,by="X1") %>% left_join(inter_pairs_c123,by="X1")

In [10]:
RNAres <- read_table("../result/RNA_Res/counts.gene.total.format.tsv")
featureStat <- cbind(as.data.frame(colSums(RNAres %>% select(-gene))),colSums(RNAres %>% select(-gene) != 0)) %>% rownames_to_column("X1")


── Column specification ────────────────────────────────────────────────────────
cols(
  gene = col_character(),
  ACEC001 = col_double(),
  ACEC002 = col_double(),
  ACEC003 = col_double(),
  ACEC004 = col_double(),
  ACEC005 = col_double(),
  AcACEC = col_double()
)



In [11]:
RNAresExon <- read_table("../result/RNA_Res/counts.exon.total.format.tsv")
featureStatExon <- cbind(as.data.frame(colSums(RNAresExon %>% select(-gene))),colSums(RNAresExon %>% select(-gene) != 0))  %>% rownames_to_column("X1")


── Column specification ────────────────────────────────────────────────────────
cols(
  gene = col_character(),
  ACEC001 = col_double(),
  ACEC002 = col_double(),
  ACEC003 = col_double(),
  ACEC004 = col_double(),
  ACEC005 = col_double(),
  AcACEC = col_double()
)



In [12]:
#是否区分父本母本
config$if_RNA_snp_spli <- TRUE
if(config$if_RNA_snp_split){
    RNAresGenome1 <- read_table2("../result/RNA_Res/counts.gene.genome1.tsv")
    RNAresGenome2 <- read_table2("../result/RNA_Res/counts.gene.genome2.tsv")
    featureStatGenome1 <- cbind(as.data.frame(colSums(RNAresGenome1 %>% select(-gene))),colSums(RNAresGenome1 %>% select(-gene) != 0)) %>% rownames_to_column("X1")
    featureStatGenome2 <- cbind(as.data.frame(colSums(RNAresGenome2 %>% select(-gene))),colSums(RNAresGenome2 %>% select(-gene) != 0)) %>% rownames_to_column("X1")
    stat <- stat %>% left_join(featureStat,by="X1") %>% left_join(featureStatExon,by="X1") %>% left_join(featureStatGenome1,by="X1")  %>% left_join(featureStatGenome2,by="X1")
    names(stat) <- c("cellname","Rawreads","DNAreads","RNAreads","yperx","raw_pairs","pairs_dedup","pairs_clean1","pairs_clean2","pairs_clean3","inter_pairs_clean3",
                     "UMIs_gene","genes_gene","UMIs_exon","genes_exon","UMIs_gene_genome1","genes_gene_genome1","UMIs_gene_genome2","genes_gene_genome2")
} else{
    stat <- stat %>% left_join(featureStat,by="X1") %>% left_join(featureStatExon,by="X1") 
    names(stat) <- c("cellname","Rawreads","DNAreads","RNAreads","yperx","raw_pairs","pairs_dedup","pairs_clean1","pairs_clean2","pairs_clean3","inter_pairs_clean3",
                     "UMIs_gene","genes_gene","UMIs_exon","genes_exon")
}

Warning message:
“`read_table2()` was deprecated in readr 2.0.0.
Please use `read_table()` instead.”

── Column specification ────────────────────────────────────────────────────────
cols(
  gene = col_character(),
  ACEC001 = col_double(),
  ACEC002 = col_double(),
  ACEC003 = col_double(),
  ACEC004 = col_double(),
  ACEC005 = col_double(),
  AcACEC = col_double()
)

Warning message:
“`read_table2()` was deprecated in readr 2.0.0.
Please use `read_table()` instead.”

── Column specification ────────────────────────────────────────────────────────
cols(
  gene = col_character(),
  ACEC001 = col_double(),
  ACEC002 = col_double(),
  ACEC003 = col_double(),
  ACEC004 = col_double(),
  ACEC005 = col_double(),
  AcACEC = col_double()
)



In [13]:
config$if_structure <- TRUE
if(config$if_structure){
    rmsd <- read_table("../stat/rmsd.info",col_names = F) %>% rowwise() %>% mutate(X1=str_split(X1,pattern = fixed("/")),cellname = X1[1], res = str_split(X1[3], fixed("."))[[1]][2],TOP3_RMSD = X5) %>% select(cellname,res,TOP3_RMSD) %>% spread(res,TOP3_RMSD) %>% arrange(cellname)
    stat <- stat %>% left_join(rmsd)
}


── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_character(),
  X2 = col_character(),
  X3 = col_character(),
  X4 = col_character(),
  X5 = col_double()
)

Joining, by = "cellname"


In [14]:
stat

cellname,Rawreads,DNAreads,RNAreads,yperx,raw_pairs,pairs_dedup,pairs_clean1,pairs_clean2,pairs_clean3,⋯,UMIs_exon,genes_exon,UMIs_gene_genome1,genes_gene_genome1,UMIs_gene_genome2,genes_gene_genome2,1m,200k,20k,50k
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AcACEC,49478513,48187731,1283576,0.000930,14898515,11608873,98278,56494,56475,⋯,49462,21829,15271,7290,15467,7443,14.90796566,27.04997172,56.6198743,42.0438349
AcCH,54534020,54975486,32,0.000714,19627010,14387527,299030,246316,246288,⋯,NA,NA,NA,NA,NA,NA,14.34330327,25.82800893,54.5573077,40.5464101
ACEC001,10125135,9956591,137178,0.001999,3188977,750346,748629,626428,626315,⋯,1061,975,462,422,489,438,0.11974493,0.21420310,0.7215551,0.3692131
ACEC002,9796166,9611707,151343,0.001800,2744529,694469,691789,573107,572999,⋯,1255,1162,609,538,679,578,0.17437566,0.24756473,1.0275267,0.5607622
ACEC003,11976507,11613822,300105,0.000649,4217239,1165618,1151802,1041239,1040967,⋯,2523,2194,1035,878,1001,835,0.08391313,0.18529710,0.6243373,0.3194716
ACEC004,14001534,13585932,345880,0.001425,4367575,972667,969120,867698,867513,⋯,1876,1712,916,789,920,775,0.03150880,0.05826526,0.3676854,0.1792004
ACEC005,14791301,14102791,567263,0.001138,4989584,1002834,994352,881681,881515,⋯,2161,1812,925,781,1016,825,0.04878526,0.13627211,0.6145058,0.3420219
CH001,6994165,6994139,2,0.001203,2833039,923193,911189,821221,820869,⋯,NA,NA,NA,NA,NA,NA,0.08425960,0.24739357,1.1384441,0.6437287
CH002,10653674,10653655,1,0.000988,4093991,1185806,1172392,1060525,1060208,⋯,NA,NA,NA,NA,NA,NA,0.11435588,0.17081405,0.8012774,0.4135539


In [15]:
config$if_cuttag<- TRUE

if (config$if_cuttag){
    reads <- tibble("cellname"=NA)
    #TODO:数量改了
    temp <- read_table("../stat/ct.read.stat",col_names = F) %>% rowwise() %>% mutate(X1 = str_split(X1,pattern ="/")[[1]][3],X2 = X2)
    names(temp) <- c("cellname","ct_reads")
    reads <- reads %>% full_join(temp)
    reads <- reads %>% filter(!is.na(cellname))

    frag <- tibble("cellname"=NA)
    temp <- read_table("../stat/ct.frag.stat",col_names = F) %>% rowwise() %>% mutate(X1 = str_extract(X1,pattern = "(?<=\\/)[A-Za-z0-9-]+(?=\\.)"))
    
    names(temp) <- c("cellname","ct_frag")
    frag <- frag %>% full_join(temp)
    frag <- frag %>% filter(!is.na(cellname))
    reads <- reads %>% left_join(frag)

    #连接
    stat <- stat %>% full_join(reads)
}




── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_character(),
  X2 = col_double()
)

Joining, by = "cellname"

── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_character(),
  X2 = col_double()
)

Joining, by = "cellname"
Joining, by = "cellname"
Joining, by = "cellname"


In [16]:
config$if_atac <- TRUE
if (config$if_atac){
    reads <- tibble("cellname"=NA)
    #TODO:数量改了
    temp <- read_table("../stat/atac.read.stat",col_names = F) %>% rowwise() %>% mutate(X1 = str_split(X1,pattern ="/")[[1]][3],X2 = X2)
    names(temp) <- c("cellname","atac_reads")
    reads <- reads %>% full_join(temp)
    reads <- reads %>% filter(!is.na(cellname))

    frag <- tibble("cellname"=NA)
    temp <- read_table("../stat/atac.frag.stat",col_names = F) %>% rowwise() %>% mutate(X1 = str_extract(X1,pattern = "(?<=\\/)[A-Za-z0-9-]+(?=\\.)"))
    
    names(temp) <- c("cellname","atac_frag")
    frag <- frag %>% full_join(temp)
    frag <- frag %>% filter(!is.na(cellname))
    reads <- reads %>% left_join(frag)

    #连接
    stat <- stat %>% full_join(reads)
}



── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_character(),
  X2 = col_double()
)

Joining, by = "cellname"

── Column specification ────────────────────────────────────────────────────────
cols(
  X1 = col_character(),
  X2 = col_double()
)

Joining, by = "cellname"
Joining, by = "cellname"
Joining, by = "cellname"


In [17]:
#这里也改了*300/1G
plot <- stat %>% mutate(RNAreadsRatio = RNAreads/(RNAreads+DNAreads),
                        pairsPerRead = raw_pairs/DNAreads,
                        pairsValidRatio = pairs_clean3/raw_pairs,
                        interPairsRatio = inter_pairs_clean3/pairs_clean3) 
#plot[is.na(plot)]<-0

In [18]:
plot

cellname,Rawreads,DNAreads,RNAreads,yperx,raw_pairs,pairs_dedup,pairs_clean1,pairs_clean2,pairs_clean3,⋯,20k,50k,ct_reads,ct_frag,atac_reads,atac_frag,RNAreadsRatio,pairsPerRead,pairsValidRatio,interPairsRatio
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AcACEC,49478513,48187731,1283576,0.000930,14898515,11608873,98278,56494,56475,⋯,56.6198743,42.0438349,1328401,848792,34618073,16130333,2.594587e-02,0.3091765,0.003790646,0.03128818
AcCH,54534020,54975486,32,0.000714,19627010,14387527,299030,246316,246288,⋯,54.5573077,40.5464101,34917133,14940931,39,32,5.820773e-07,0.3570139,0.012548422,0.04373335
ACEC001,10125135,9956591,137178,0.001999,3188977,750346,748629,626428,626315,⋯,0.7215551,0.3692131,142440,8406,4053722,162579,1.359036e-02,0.3202880,0.196399974,0.39476462
ACEC002,9796166,9611707,151343,0.001800,2744529,694469,691789,573107,572999,⋯,1.0275267,0.5607622,110153,9488,5598230,360234,1.550161e-02,0.2855402,0.208778628,0.26732507
ACEC003,11976507,11613822,300105,0.000649,4217239,1165618,1151802,1041239,1040967,⋯,0.6243373,0.3194716,303277,15460,5732338,213615,2.518943e-02,0.3631224,0.246836141,0.29681440
ACEC004,14001534,13585932,345880,0.001425,4367575,972667,969120,867698,867513,⋯,0.3676854,0.1792004,216809,10120,7100575,217592,2.482663e-02,0.3214778,0.198625782,0.33923757
ACEC005,14791301,14102791,567263,0.001138,4989584,1002834,994352,881681,881515,⋯,0.6145058,0.3420219,639621,19519,4073894,103050,3.866809e-02,0.3538012,0.176671041,0.33342144
CH001,6994165,6994139,2,0.001203,2833039,923193,911189,821221,820869,⋯,1.1384441,0.6437287,2252536,194568,4,0,2.859536e-07,0.4050590,0.289748570,0.20312376
CH002,10653674,10653655,1,0.000988,4093991,1185806,1172392,1060525,1060208,⋯,0.8012774,0.4135539,4738863,369097,6,6,9.386449e-08,0.3842804,0.258966861,0.24884174


In [19]:
plot %>% write_tsv("summary.stat.tsv")

In [11]:
options(repr.matrix.max.cols=50, repr.matrix.max.rows=100)
plot %>% summary()

   cellname            Rawreads         DNAreads         RNAreads       
 Length:192         Min.   :0.3023   Min.   :0.2973   Min.   :0.002575  
 Class :character   1st Qu.:2.4015   1st Qu.:2.2635   1st Qu.:0.046244  
 Mode  :character   Median :4.1420   Median :3.8554   Median :0.152186  
                    Mean   :4.0641   Mean   :3.8001   Mean   :0.230230  
                    3rd Qu.:5.5827   3rd Qu.:5.1283   3rd Qu.:0.336940  
                    Max.   :8.5390   Max.   :8.1782   Max.   :1.275965  
     yperx            raw_pairs        pairs_dedup      pairs_clean1   
 Min.   :0.000965   Min.   :  65874   Min.   : 51698   Min.   : 51559  
 1st Qu.:0.005006   1st Qu.: 662960   1st Qu.:307964   1st Qu.:305150  
 Median :0.005871   Median :1322358   Median :373257   Median :368597  
 Mean   :0.007437   Mean   :1363911   Mean   :375294   Mean   :371832  
 3rd Qu.:0.006839   3rd Qu.:1912474   3rd Qu.:448309   3rd Qu.:444313  
 Max.   :0.095524   Max.   :3351396   Max.   :671798   Ma